In [0]:
cosmosendpoint = "https://fabianm-oltp-spark-workshop-cdb.documents.azure.com:443/"
cosmosmasterkey = ""
cosmosserviceprincipalpassword=""


In [0]:
import base64
import os
cert_file= open("/workspace/users/fabianm@microsoft.com/fabianm-spark-auth-sp-cert.pem","rb")
cert_data_binary = cert_file.read()
cert_data = (base64.b64encode(cert_data_binary)).decode('ascii')


In [0]:
import random
import string
import uuid
from pyspark.sql.types import structtype,structfield, stringtype, integertype
 
def random_string_generator(str_size, allowed_chars):
    return ''.join(random.choice(allowed_chars) for x in range(str_size))
 
pk = str(uuid.uuid4())    
print("pk: " + pk)
    
chars = string.ascii_letters
data = [\
    (str(uuid.uuid4()), pk, "", none, 0, random_string_generator(16000, chars)),\
    (str(uuid.uuid4()), pk, "", none, 0, random_string_generator(random.randint(16000, 170000), chars)),\
  ]

schema = structtype([ \
    structfield("id",stringtype(),false), \
    structfield("pk",stringtype(),false), \
    structfield("emptycolumn",stringtype(),true), \
    structfield("nullcolumn",stringtype(),true), \
    structfield("defaultcolumn",integertype(),true), \
    structfield("largecolumn", stringtype(), true)\
  ])
 
df = spark.createdataframe(data=data,schema=schema)
df.printschema()
df.show(truncate=100)

writecfg = {
  "spark.cosmos.accountendpoint": cosmosendpoint,
  "spark.cosmos.database": "test",
  "spark.cosmos.container": "testitems",
  "spark.cosmos.write.strategy": "itemappend",
  "spark.cosmos.write.bulk.enabled": "true",  
  "cosmos.auth.sample.enabled": "true",
  # masterkey
  #"cosmos.auth.sample.authtype": "masterkey",
  #"cosmos.auth.sample.key.secret": cosmosmasterkey,
  #
  # aad auth with managed identity
  #"cosmos.auth.sample.authtype": "managedidentity",
  #"cosmos.auth.sample.tenantid": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  #"cosmos.auth.sample.subscriptionid": "8fba6d4f-7c37-4d13-9063-fd58ad2b86e2",
  #"cosmos.auth.sample.resourcegroupname": "fabianm-oltp-spark-workshop"
  #
  # aad auth with service principal (password)
  #"cosmos.auth.sample.authtype": "serviceprincipal",
  #"cosmos.auth.sample.tenantid": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  #"cosmos.auth.sample.subscriptionid": "8fba6d4f-7c37-4d13-9063-fd58ad2b86e2",
  #"cosmos.auth.sample.resourcegroupname": "fabianm-oltp-spark-workshop",
  #"cosmos.auth.sample.serviceprincipal.clientid": "bd559cf4-786d-43ae-9ff6-eb83c5952c73",
  #"cosmos.auth.sample.serviceprincipal.clientsecret": cosmosserviceprincipalpassword
  #
  # aad auth with service principal (cert)
  #"cosmos.auth.sample.authtype": "serviceprincipal",
  #"cosmos.auth.sample.tenantid": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  #"cosmos.auth.sample.subscriptionid": "8fba6d4f-7c37-4d13-9063-fd58ad2b86e2",
  #"cosmos.auth.sample.resourcegroupname": "fabianm-oltp-spark-workshop",
  #"cosmos.auth.sample.serviceprincipal.clientid": "88436299-945f-4824-8183-2cbddf981388",
  #"cosmos.auth.sample.serviceprincipal.cert": cert_data
}

df\
  .write \
  .format("cosmos.oltp") \
  .mode("append") \
  .options(**writecfg) \
  .save()


pk: 7d90716b-4ea1-4753-8090-4b72e4a2b93b
root
 |-- id: string (nullable = false)
 |-- pk: string (nullable = false)
 |-- emptycolumn: string (nullable = true)
 |-- nullcolumn: string (nullable = true)
 |-- defaultcolumn: integer (nullable = true)
 |-- largecolumn: string (nullable = true)

+------------------------------------+------------------------------------+-----------+----------+-------------+----------------------------------------------------------------------------------------------------+
|                                  id|                                  pk|emptycolumn|nullcolumn|defaultcolumn|                                                                                         largecolumn|
+------------------------------------+------------------------------------+-----------+----------+-------------+----------------------------------------------------------------------------------------------------+
|fa1a3854-4d41-4ffb-b992-a00c82585ddc|7d90716b-4ea1-4753-8090-4b72e